# Capstone Project: Lighthouse clustering

## Introduction/Business Problem

Galicia is a spanish region with many kilometers of coastline. In order to avoid maritime accidents, along the coastline there are plenty of lighthouses to guide the ships. These places are usually isolated and are far from touristic areas.

The Galician government has a plan to boost tourism in the villages near these lighthouses: they will subsidize newly opened catering businesses located less than 3 kilometers from a lighthouse. A key fact is that the amount of money given to a new business will be proportional to the age of the lighthouse, since this feature will increase the tourist attraction of that place.

A company is interested in creating a new catering business, fulfilling the requirements to receive the grant. The problem is knowing which is the best lighthouse to place the food business, because within a radius of 3 km from many of the Galician lighthouses there are already restaurants and bars.

The purpose of the analysis will be finding the best place to install the catering business, according to the restaurant density in that area and the age of the lighthouse.

Let's import the necessary libraries for data analysis

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd
 # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Data source, clean up and selection

We can find a list of the main spanish lighthouses at <a href="https://es.wikipedia.org/wiki/Anexo:Faros_de_Espa%C3%B1a#Lista_de_faros" target="_blank">Wikipedia</a>. This table contains, apart from the name and coordinates of each lighthouse, other features such as height, light range or year of construction. Some of these features will be used during the data analysis, in particular coordinates and built year.

In [2]:
lh = pd.read_html("https://es.wikipedia.org/wiki/Anexo:Faros_de_Espa%C3%B1a", header = 0)[1]

Let's change the column names to English:

In [3]:
col_names = ["Lighthouse Name", "Image", "Location&Coordinates", "Province/Community", "Autority", "WaterBody", "YearBuilt", "TowerHeight", "FocalHeight", "Range"]
lh.columns = col_names
lh.head()

,Lighthouse Name,Image,Location&Coordinates,Province/Community,Autority,WaterBody,YearBuilt,TowerHeight,FocalHeight,Range
0,Adra[6]​,NaN,"Adra36°44′53″N 3°01′52″O﻿ / ﻿36.74796, -3.03098",AlmeríaAndalucía,Almería,Mar Mediterráneo,1889,26 metros (85 pies),49 metros (161 pies),16 millas náuticas (30 km)
1,Águilas[7]​,NaN,"Águilas37°24′06″N 1°34′41″O﻿ / ﻿37.40169, -1.5...",Murcia,Cartagena,Mar Mediterráneo,1860,23 metros (75 pies),30 metros (98 pies),13 millas náuticas (24 km)
2,Ahorcados[8]​[9]​,NaN,Isla de los Ahorcados38°48′52″N 1°24′42″E﻿ / ﻿...,Islas Baleares,Baleares,Mar Mediterráneo,1856,17 metros (56 pies),27 metros (89 pies),10 millas náuticas (19 km)
3,Albir[10]​[11]​,NaN,"Alfaz del Pi38°33′49″N 0°03′00″O﻿ / ﻿38.56363,...",AlicanteComunidad Valenciana,Alicante,Mar Mediterráneo,1863,8 metros (26 pies),112 metros (367 pies),15 millas náuticas (28 km)
4,Alcanada[12]​[13]​,NaN,"Mallorca39°50′07″N 3°10′16″E﻿ / ﻿39.83541, 3.1...",Islas Baleares,Baleares,Mar Mediterráneo,1861,15 metros (49 pies),25 metros (82 pies),11 millas náuticas (20 km)


It is necessary to make an important effort to clean up this data table, so that the final format is readable for us. First of all, let's drop some columns that are not of interest for the analysis:

In [4]:
lh.drop(["Image", "Autority", "WaterBody"], axis = 1, inplace = True)
lh.tail()

,Lighthouse Name,Location&Coordinates,Province/Community,YearBuilt,TowerHeight,FocalHeight,Range
187,Torre de Hércules[329]​[330]​,"La Coruña43°23′09″N 8°24′23″O﻿ / ﻿43.38594, -8...",La CoruñaGalicia,Siglo II,49 metros (161 pies),106 metros (348 pies),23 millas náuticas (43 km)
188,Tramuntana[331]​[332]​,Isla Dragonera39°35′53″N 2°20′17″E﻿ / ﻿39.5980...,Islas Baleares,1910,15 metros (49 pies),67 metros (220 pies),14 millas náuticas (26 km)
189,Valencia[333]​[334]​,Puerto de Valencia39°26′57″N 0°18′08″O﻿ / ﻿39....,ValenciaComunidad Valenciana,1909,22 metros (72 pies),30 metros (98 pies),20 millas náuticas (37 km)
190,Valencia (2015)[335]​[nota 1]​[336]​,Puerto de Valencia39°27′18″N 0°17′10″O﻿ / ﻿39....,ValenciaComunidad Valenciana,2015,32 metros (105 pies),NaN,25 metros (82 pies)
191,Zumaya[337]​[338]​,"Zumaya43°18′08″N 2°15′04″O﻿ / ﻿43.30216, -2.25102",GuipúzcoaPaís Vasco,1882,12 metros (39 pies),41 metros (135 pies),14 millas náuticas (26 km)


In the previous dataframe we observe that row 190 has a NaN at FocalHeight column. Let's drop this row as it does not belong to the region of Galicia, so it will not be of interest in the analysis:

In [5]:
lh.drop(190, axis = 0, inplace = True)
lh.reset_index(drop = True, inplace = True)
lh.tail()

,Lighthouse Name,Location&Coordinates,Province/Community,YearBuilt,TowerHeight,FocalHeight,Range
186,Touriñán (Toriñana)[327]​[328]​,Cabo Touriñán43°03′12″N 9°17′53″O﻿ / ﻿43.05336...,La CoruñaGalicia,1898,14 metros (46 pies),65 metros (213 pies),24 millas náuticas (44 km)
187,Torre de Hércules[329]​[330]​,"La Coruña43°23′09″N 8°24′23″O﻿ / ﻿43.38594, -8...",La CoruñaGalicia,Siglo II,49 metros (161 pies),106 metros (348 pies),23 millas náuticas (43 km)
188,Tramuntana[331]​[332]​,Isla Dragonera39°35′53″N 2°20′17″E﻿ / ﻿39.5980...,Islas Baleares,1910,15 metros (49 pies),67 metros (220 pies),14 millas náuticas (26 km)
189,Valencia[333]​[334]​,Puerto de Valencia39°26′57″N 0°18′08″O﻿ / ﻿39....,ValenciaComunidad Valenciana,1909,22 metros (72 pies),30 metros (98 pies),20 millas náuticas (37 km)
190,Zumaya[337]​[338]​,"Zumaya43°18′08″N 2°15′04″O﻿ / ﻿43.30216, -2.25102",GuipúzcoaPaís Vasco,1882,12 metros (39 pies),41 metros (135 pies),14 millas náuticas (26 km)


Next step consists of extracting readable data from Lighthouse Name and Location&Coordinates column. We get this by properly spliting the entries on each column:

In [6]:
lightlist = []
latlist = []
longlist = []
for lighthouse, locationcoordinates in zip(lh["Lighthouse Name"], lh["Location&Coordinates"]):
    lighthouse = lighthouse.split("[")[0]
    lightlist.append(lighthouse)

    latitude = locationcoordinates.split("/ \ufeff")[1].split(", ")[0]
    longitude = locationcoordinates.split("/ \ufeff")[1].split(", ")[1]
    latlist.append(latitude)
    longlist.append(longitude)
    
lh["Lighthouse Name"] = lightlist
lh.insert(1, "Latitude", latlist)
lh.insert(2, "Longitude", longlist)

lh.drop("Location&Coordinates", axis = 1, inplace = True)
lh.head()

,Lighthouse Name,Latitude,Longitude,Province/Community,YearBuilt,TowerHeight,FocalHeight,Range
0,Adra,36.74796,-3.03098,AlmeríaAndalucía,1889,26 metros (85 pies),49 metros (161 pies),16 millas náuticas (30 km)
1,Águilas,37.40169,-1.57802,Murcia,1860,23 metros (75 pies),30 metros (98 pies),13 millas náuticas (24 km)
2,Ahorcados,38.81451,1.41178,Islas Baleares,1856,17 metros (56 pies),27 metros (89 pies),10 millas náuticas (19 km)
3,Albir,38.56363,-0.05007,AlicanteComunidad Valenciana,1863,8 metros (26 pies),112 metros (367 pies),15 millas náuticas (28 km)
4,Alcanada,39.83541,3.17117,Islas Baleares,1861,15 metros (49 pies),25 metros (82 pies),11 millas náuticas (20 km)


At the next stage, in the columns TowerHeight, FocalHeight and Range we will only retain numeric data (in meters and nautical miles, respectively):

In [7]:
towerlist = []
focallist = []
range2list = []
for towerh, focalh, range2 in zip(lh["TowerHeight"], lh["FocalHeight"], lh["Range"]):
    towerh = towerh.split("\xa0metros")[0]
    towerlist.append(towerh)

    focalh = focalh.split("\xa0metros")[0]
    focallist.append(focalh)
    
    range2 = range2.split("\xa0millas")[0]
    range2list.append(range2)
    
lh["TowerHeight"] = towerlist
lh["FocalHeight"] = focallist
lh["Range"] = range2list

lh.tail()

,Lighthouse Name,Latitude,Longitude,Province/Community,YearBuilt,TowerHeight,FocalHeight,Range
186,Touriñán (Toriñana),43.05336,-9.29814,La CoruñaGalicia,1898,14,65,24
187,Torre de Hércules,43.38594,-8.40648,La CoruñaGalicia,Siglo II,49,106,23
188,Tramuntana,39.59803,2.33818,Islas Baleares,1910,15,67,14
189,Valencia,39.44910,-0.30223,ValenciaComunidad Valenciana,1909,22,30,20
190,Zumaya,43.30216,-2.25102,GuipúzcoaPaís Vasco,1882,12,41,14


As we can see in the previous tail visualization, there will be some problems when converting object data to float data, in particular in YearBuilt column. Let's see which are the problematic rows when converting to float format:

In [8]:
#Convert numerical features from string to float. Print the rows where there is any problem
problem_list = []
for i in range(len(lh["YearBuilt"])):
    try:
        lh.loc[i, "YearBuilt"] = float(lh.loc[i, "YearBuilt"])
        lh.loc[i, "TowerHeight"] = float(lh.loc[i, "TowerHeight"])
        lh.loc[i, "FocalHeight"] = float(lh.loc[i, "FocalHeight"])
        lh.loc[i, "Range"] = float(lh.loc[i, "Range"])
    except:
        problem_list.append(i)
lh.loc[problem_list,:]

,Lighthouse Name,Latitude,Longitude,Province/Community,YearBuilt,TowerHeight,FocalHeight,Range
21,Cabo Tortosa,40.7155,0.92847,TarragonaCataluña,1864/1984,18,18,14
45,Cabo Roche,36.29553,-6.13992,CádizAndalucía,Siglo XVI,20,45,20
50,Cabo Villano,43.16041,-9.21093,La CoruñaGalicia,1854/1896,25,104,28
51,Carbonera,36.24452,-5.30129,CádizAndalucía,Siglo XVII,14,39,14
106,Mouro,43.47328,-3.75585,Cantabria,1860,"18,70","41,50",11
187,Torre de Hércules,43.38594,-8.40648,La CoruñaGalicia,Siglo II,49,106,23


The previous rows have some inadequate entries to be converted to float. We will fix this problems manually:

In [9]:
#Let's fix every problem separately:
#If we have two distinct construction years, we choose the last one, as it's more representative of the current lighthouse
lh.loc[21, "YearBuilt"] = float(1984)
lh.loc[50, "YearBuilt"] = float(1896)

#If YearBuilt is not a real year but a century, the middle year of that century is chosen
lh.loc[45, "YearBuilt"] = float(1550)
lh.loc[51, "YearBuilt"] = float(1650)
lh.loc[187, "YearBuilt"] = float(150)

#Problem with comma separator
lh.loc[106, "TowerHeight"] = float(18.70)
lh.loc[106, "FocalHeight"] = float(41.50)

Now the conversion will be successful:

In [10]:
lh["Latitude"] = lh["Latitude"].astype(float)
lh["Longitude"] = lh["Longitude"].astype(float)

lh["YearBuilt"] = lh["YearBuilt"].astype(float)
lh["TowerHeight"] = lh["TowerHeight"].astype(float)
lh["FocalHeight"] = lh["FocalHeight"].astype(float)
lh["Range"] = lh["Range"].astype(float)
lh.dtypes

Lighthouse Name        object
Latitude              float64
Longitude             float64
Province/Community     object
YearBuilt             float64
TowerHeight           float64
FocalHeight           float64
Range                 float64
dtype: object

In [11]:
lh.head(10)

,Lighthouse Name,Latitude,Longitude,Province/Community,YearBuilt,TowerHeight,FocalHeight,Range
0,Adra,36.74796,-3.03098,AlmeríaAndalucía,1889.0,26.0,49.0,16.0
1,Águilas,37.40169,-1.57802,Murcia,1860.0,23.0,30.0,13.0
2,Ahorcados,38.81451,1.41178,Islas Baleares,1856.0,17.0,27.0,10.0
3,Albir,38.56363,-0.05007,AlicanteComunidad Valenciana,1863.0,8.0,112.0,15.0
4,Alcanada,39.83541,3.17117,Islas Baleares,1861.0,15.0,25.0,11.0
5,Almería (San Telmo),36.82845,-2.49231,AlmeríaAndalucía,1865.0,7.0,77.0,19.0
6,Arenas Blancas,28.56992,-17.76056,Santa Cruz de TenerifeIslas Canarias,1997.0,38.0,46.0,20.0
7,Avilés,43.59559,-5.94533,Asturias,1863.0,14.0,40.0,20.0
8,Bleda Plana,38.97984,1.15902,Islas Baleares,1927.0,8.0,28.0,10.0
9,Botafoc,38.90416,1.45388,Islas Baleares,1861.0,16.0,31.0,14.0


Remember that our analysis will focus on lighthouses placed in Galicia, which is named in Province/Community column. In Spain, a province is a member of a Community, and a Community is the main division of the country. So let's properly separate province and community and finally drop provinces:

In [12]:
col_pc = lh["Province/Community"]
for i in range(len(col_pc)):
    for letter in range(len(col_pc[i])-1):
        if (col_pc[i][letter].islower()) and (col_pc[i][letter+1].isupper()):
            lh.loc[i,"Province/Community"] = col_pc[i].split(col_pc[i][letter-2] + col_pc[i][letter-1] + col_pc[i][letter])[1]
            break
lh.tail()

,Lighthouse Name,Latitude,Longitude,Province/Community,YearBuilt,TowerHeight,FocalHeight,Range
186,Touriñán (Toriñana),43.05336,-9.29814,Galicia,1898.0,14.0,65.0,24.0
187,Torre de Hércules,43.38594,-8.40648,Galicia,150.0,49.0,106.0,23.0
188,Tramuntana,39.59803,2.33818,Islas Baleares,1910.0,15.0,67.0,14.0
189,Valencia,39.44910,-0.30223,Comunidad Valen,1909.0,22.0,30.0,20.0
190,Zumaya,43.30216,-2.25102,País Vasco,1882.0,12.0,41.0,14.0


As we see in the above dataframe visualization, "Comunidad Valen" and "Comunidad Valencia" make reference to the same community. Let's join the corresponding entries in a single category:

In [13]:
#"Comunidad Valen" makes reference to the same community as "Comunidad Valencia", so let's join them
lh[lh["Province/Community"] == "Comunidad Valen"]

,Lighthouse Name,Latitude,Longitude,Province/Community,YearBuilt,TowerHeight,FocalHeight,Range
11,Cabo Canet,39.67449,-0.20765,Comunidad Valen,1904.0,30.0,33.0,20.0
59,Cullera,39.18651,-0.21697,Comunidad Valen,1858.0,16.0,28.0,19.0
189,Valencia,39.44910,-0.30223,Comunidad Valen,1909.0,22.0,30.0,20.0


In [14]:
lh.loc[[11,59,189], "Province/Community"] = "Comunidad Valenciana"

At the following line, we can see a ranking of Communities by number of lighthouses:

In [15]:
lh["Province/Community"].value_counts()

Islas Baleares          33
Andalucía               31
Islas Canarias          27
Galicia                 24
Cataluña                17
Asturias                16
Comunidad Valenciana    15
País Vasco              10
Murcia                   7
Cantabria                7
Melilla                  3
Ceuta                    1
Name: Province/Community, dtype: int64

Now we have a clean dataframe containing all spanish lighthouses, their coordinates, community, year of construction, tower and focal height and light range. We forgot to rename column from Province/Community to Community. Let's do it now:

In [16]:
lh.rename(columns={"Province/Community":"Community"}, inplace = True)
lh.head()

,Lighthouse Name,Latitude,Longitude,Community,YearBuilt,TowerHeight,FocalHeight,Range
0,Adra,36.74796,-3.03098,Andalucía,1889.0,26.0,49.0,16.0
1,Águilas,37.40169,-1.57802,Murcia,1860.0,23.0,30.0,13.0
2,Ahorcados,38.81451,1.41178,Islas Baleares,1856.0,17.0,27.0,10.0
3,Albir,38.56363,-0.05007,Comunidad Valenciana,1863.0,8.0,112.0,15.0
4,Alcanada,39.83541,3.17117,Islas Baleares,1861.0,15.0,25.0,11.0


Once we have cleaned our date, let's focus on Galician lighthouses:

In [17]:
#Our research will be centered in Galicia. Let's only select the lighthouses belonging to this community
lh_gal = lh[lh["Community"] == "Galicia"]
lh_gal.reset_index(drop = True, inplace = True)
lh_gal.head()

,Lighthouse Name,Latitude,Longitude,Community,YearBuilt,TowerHeight,FocalHeight,Range
0,Cabo Corrubedo,42.57627,-9.09009,Galicia,1854.0,14.0,32.0,15.0
1,Cabo Finisterre,42.88236,-9.27196,Galicia,1853.0,17.0,143.0,23.0
2,Cabo Ortegal,43.77107,-7.87018,Galicia,1954.0,10.0,124.0,18.0
3,Cabo Prior,43.56761,-8.31453,Galicia,1853.0,7.0,107.0,22.0
4,Cabo Prioriño,43.45879,-8.34033,Galicia,1854.0,5.0,36.0,23.0


## Foursquare data: most common venues

It is time to use Foursquare API to find out the nearest restaurants and bars from every lighthouse. For that, we will only look for food venues, our search will be limited to 10 venues per lighthouse within a radius of 3 km.

Let's find out the coordinates of Antas de Ulla, a location which is approximately the geographical center of Galicia. We do this thinking on future map plots:

In [18]:
address = 'Antas de Ulla, Galicia, Spain'
geolocator = Nominatim(user_agent="au_explorer", timeout=3)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Antas de Ulla are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Antas de Ulla are 42.7674234, -7.8984065.


Let's introduce our Foursquare credentials for future searches:

In [19]:
#Foursquare credentials
CLIENT_ID = 'Z1VI0IH3N3P3JFJM1WJ1A03CGG3LWLTVO5M4YDVK41G1VJ3L' # your Foursquare ID
CLIENT_SECRET = 'UT001UYRQLPB5SOI2CDDQUUNPFAGXJJJJZHYB31HZAKDRI4J' # your Foursquare Secret
VERSION = '20191026' #Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z1VI0IH3N3P3JFJM1WJ1A03CGG3LWLTVO5M4YDVK41G1VJ3L
CLIENT_SECRET:UT001UYRQLPB5SOI2CDDQUUNPFAGXJJJJZHYB31HZAKDRI4J


The following function gives us the nearby venues to a given location, as we already did in previous labs on this course. Note that in the url we incluid the key "section", with the assigned item "food", so we only look for this kind of venues.

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000, LIMIT = 10):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            "food")
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Lighthouse Name', 
                  'Lighthouse Latitude', 
                  'Lighthouse Longitude', 
                  'Venue Name', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
lighthouse_venues = getNearbyVenues(names=lh_gal['Lighthouse Name'],
                                   latitudes=lh_gal['Latitude'],
                                   longitudes=lh_gal['Longitude']
                                  )

Cabo Corrubedo
Cabo Finisterre
Cabo Ortegal
Cabo Prior
Cabo Prioriño
Cabo Silleiro
Cabo Villano
Estaca de Bares
Isla de Sálvora
Isla de Ons
Isla Pancha
Islas Sisargas
La Guía
Monte del Faro
Punta Atalaya (o San Ciprián)
Punta Cabalo
Punta Candelaria
Punta Frouxeira
Punta Insua
Punta Laxe
Punta Nariga
Punta Roncadoira
Touriñán (Toriñana)
Torre de Hércules


As we can see in the next table, in Galicia there are some lighthouses with more than 10 venues in a 3 km radius, but there are some others with no venues at all as they are not present in the table:

In [22]:
lighthouse_venues["Lighthouse Name"].value_counts()

Cabo Finisterre                  10
Isla Pancha                      10
Punta Laxe                       10
Torre de Hércules                10
La Guía                          10
Punta Frouxeira                   5
Punta Atalaya (o San Ciprián)     5
Isla de Ons                       5
Punta Cabalo                      4
Monte del Faro                    4
Cabo Corrubedo                    4
Cabo Silleiro                     4
Estaca de Bares                   3
Cabo Ortegal                      2
Punta Insua                       1
Cabo Prioriño                     1
Name: Lighthouse Name, dtype: int64

Before we add the non-venue missing lighthouses to our dataset, let's do one hot encoding to the lighthouses in "lighthouse_venues", so we can see clearly what kind of venues are there in each place:

In [23]:
# one hot encoding
lighthouse_onehot = pd.get_dummies(lighthouse_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
lighthouse_onehot["Lighthouse Name"] = lighthouse_venues["Lighthouse Name"]

# move neighborhood column to the first column
fixed_columns = [lighthouse_onehot.columns[-1]] + list(lighthouse_onehot.columns[:-1])
lighthouse_onehot = lighthouse_onehot[fixed_columns]

lighthouse_onehot.head()

,Lighthouse Name,Bakery,Brazilian Restaurant,Breakfast Spot,Burger Joint,Cafeteria,Café,Diner,Food,Food Court,Gastropub,Italian Restaurant,Mexican Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Spanish Restaurant,Tapas Restaurant,Turkish Restaurant
0,Cabo Corrubedo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Cabo Corrubedo,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Cabo Corrubedo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Cabo Corrubedo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,Cabo Finisterre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Now let's see the frequency of each venue for grouped lighthouses:

In [24]:
lighthouse_grouped = lighthouse_onehot.groupby('Lighthouse Name').mean().reset_index()
lighthouse_grouped

,Lighthouse Name,Bakery,Brazilian Restaurant,Breakfast Spot,Burger Joint,Cafeteria,Café,Diner,Food,Food Court,Gastropub,Italian Restaurant,Mexican Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Spanish Restaurant,Tapas Restaurant,Turkish Restaurant
0,Cabo Corrubedo,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.25,0.0,0.000000,0.00,0.250000,0.5,0.0
1,Cabo Finisterre,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.1,0.0,0.00,0.0,0.0,0.1,0.10,0.1,0.400000,0.00,0.100000,0.0,0.0
2,Cabo Ortegal,0.0,0.0,0.0,0.0,0.00,0.0,0.5,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.500000,0.00,0.000000,0.0,0.0
3,Cabo Prioriño,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.000000,0.0,1.0
4,Cabo Silleiro,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.50,0.0,0.250000,0.00,0.000000,0.0,0.0
5,Estaca de Bares,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.666667,0.00,0.333333,0.0,0.0
6,Isla Pancha,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.1,0.0,0.00,0.1,0.0,0.1,0.10,0.0,0.100000,0.00,0.400000,0.0,0.0
7,Isla de Ons,0.0,0.0,0.0,0.2,0.00,0.0,0.0,0.4,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.200000,0.00,0.000000,0.2,0.0
8,La Guía,0.0,0.1,0.1,0.1,0.00,0.1,0.0,0.0,0.0,0.00,0.0,0.1,0.0,0.10,0.0,0.200000,0.00,0.200000,0.0,0.0
9,Monte del Faro,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.25,0.0,0.250000,0.25,0.250000,0.0,0.0


The following step is very important: the addition of the lighthouses with no venues to the previous list. Without this process, we will not be able to carry out a satisfactory clustering analysis, as we are just interested in lighthouses with low density of food venues. Let's add a zero row for each lighthouse with no venues at all:

In [25]:
#Add lighthouses with no food items
for lighthouse in lh_gal["Lighthouse Name"]:
    if lighthouse not in lighthouse_grouped["Lighthouse Name"].values:
        zerovalues = [lighthouse]
        for i in range(len(lighthouse_grouped.columns[1:])):
            zerovalues.append([0])
        lighthousedf = pd.DataFrame(dict(zip(lighthouse_grouped.columns, zerovalues)))
        lighthouse_grouped = lighthouse_grouped.append(lighthousedf, ignore_index = True)
lighthouse_grouped_sorted = lighthouse_grouped.sort_values("Lighthouse Name")
lighthouse_grouped_sorted.reset_index(drop = True, inplace = True)
lighthouse_grouped_sorted

,Lighthouse Name,Bakery,Brazilian Restaurant,Breakfast Spot,Burger Joint,Cafeteria,Café,Diner,Food,Food Court,Gastropub,Italian Restaurant,Mexican Restaurant,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Spanish Restaurant,Tapas Restaurant,Turkish Restaurant
0,Cabo Corrubedo,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.25,0.0,0.000000,0.00,0.250000,0.5,0.0
1,Cabo Finisterre,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.1,0.0,0.00,0.0,0.0,0.1,0.10,0.1,0.400000,0.00,0.100000,0.0,0.0
2,Cabo Ortegal,0.0,0.0,0.0,0.0,0.00,0.0,0.5,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.500000,0.00,0.000000,0.0,0.0
3,Cabo Prior,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.000000,0.0,0.0
4,Cabo Prioriño,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.000000,0.0,1.0
5,Cabo Silleiro,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.50,0.0,0.250000,0.00,0.000000,0.0,0.0
6,Cabo Villano,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.000000,0.00,0.000000,0.0,0.0
7,Estaca de Bares,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.666667,0.00,0.333333,0.0,0.0
8,Isla Pancha,0.0,0.0,0.0,0.0,0.00,0.0,0.1,0.1,0.0,0.00,0.1,0.0,0.1,0.10,0.0,0.100000,0.00,0.400000,0.0,0.0
9,Isla de Ons,0.0,0.0,0.0,0.2,0.00,0.0,0.0,0.4,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.200000,0.00,0.000000,0.2,0.0


Function that returns the most common venues for each lighthouse:

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories[row_categories[:] !=0].sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

We return the 10 most common venues, it they exist, for each lighthouse:

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Lighthouse Name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Lighthouse Name'] = lighthouse_grouped_sorted['Lighthouse Name']

for ind in np.arange(lighthouse_grouped_sorted.shape[0]):
    length = len(return_most_common_venues(lighthouse_grouped_sorted.iloc[ind, :], num_top_venues))
    neighborhoods_venues_sorted.iloc[ind, 1:length+1] = return_most_common_venues(lighthouse_grouped_sorted.iloc[ind, :], num_top_venues)
    neighborhoods_venues_sorted.iloc[ind, length+1:] = (num_top_venues-length)*["None"]        
neighborhoods_venues_sorted

,Lighthouse Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cabo Corrubedo,Tapas Restaurant,Spanish Restaurant,Restaurant,None,None,None,None,None,None,None
1,Cabo Finisterre,Seafood Restaurant,Spanish Restaurant,Sandwich Place,Restaurant,Pizza Place,Food,Diner,None,None,None
2,Cabo Ortegal,Seafood Restaurant,Diner,None,None,None,None,None,None,None,None
3,Cabo Prior,None,None,None,None,None,None,None,None,None,None
4,Cabo Prioriño,Turkish Restaurant,None,None,None,None,None,None,None,None,None
5,Cabo Silleiro,Restaurant,Seafood Restaurant,Cafeteria,None,None,None,None,None,None,None
6,Cabo Villano,None,None,None,None,None,None,None,None,None,None
7,Estaca de Bares,Seafood Restaurant,Spanish Restaurant,None,None,None,None,None,None,None,None
8,Isla Pancha,Spanish Restaurant,Seafood Restaurant,Restaurant,Pizza Place,Italian Restaurant,Food,Diner,None,None,None
9,Isla de Ons,Food,Tapas Restaurant,Seafood Restaurant,Burger Joint,None,None,None,None,None,None


## Clustering lighthouses

First of all, let's display the lighthouses in the map of Galicia:

In [28]:
map_faros = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, label in zip(lh_gal['Latitude'], lh_gal['Longitude'], lh_gal['Lighthouse Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_faros)  
    
map_faros

At the following, we will cluster the lighthouses according to the most common venues around them. For that purpose, let's use kMeans algorithm.

In [29]:
# set number of clusters
kclusters = 6

lighthouse_grouped_clustering = lighthouse_grouped_sorted.drop('Lighthouse Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lighthouse_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 5, 3, 1, 4, 5, 1, 0, 2, 5])

In [30]:
# add clustering labels
#neighborhoods_venues_sorted.drop('Cluster Labels', axis = 1, inplace = True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

lighthouse_merged = lh_gal

# merge lighthouse_merged with lh_gal to add latitude/longitude for each neighborhood
lighthouse_merged = lighthouse_merged.merge(neighborhoods_venues_sorted, on = "Lighthouse Name", how = "left")

lighthouse_merged.head() # check the last columns!

,Lighthouse Name,Latitude,Longitude,Community,YearBuilt,TowerHeight,FocalHeight,Range,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cabo Corrubedo,42.57627,-9.09009,Galicia,1854.0,14.0,32.0,15.0,2,Tapas Restaurant,Spanish Restaurant,Restaurant,None,None,None,None,None,None,None
1,Cabo Finisterre,42.88236,-9.27196,Galicia,1853.0,17.0,143.0,23.0,5,Seafood Restaurant,Spanish Restaurant,Sandwich Place,Restaurant,Pizza Place,Food,Diner,None,None,None
2,Cabo Ortegal,43.77107,-7.87018,Galicia,1954.0,10.0,124.0,18.0,3,Seafood Restaurant,Diner,None,None,None,None,None,None,None,None
3,Cabo Prior,43.56761,-8.31453,Galicia,1853.0,7.0,107.0,22.0,1,None,None,None,None,None,None,None,None,None,None
4,Cabo Prioriño,43.45879,-8.34033,Galicia,1854.0,5.0,36.0,23.0,4,Turkish Restaurant,None,None,None,None,None,None,None,None,None


We display the 4 cluster on a map of Galicia, so it is possible to better interpret the results:

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=8)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lighthouse_merged['Latitude'], lighthouse_merged['Longitude'], lighthouse_merged['Lighthouse Name'], lighthouse_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Once the clustering has finished, let's explore the parts of the dataset belonging to each cluster:

Cluster 0 has only two lighthouses, highlighted by its Most common venue: seafood restaurant. This kind of food is very typical of Galicia

In [32]:
lighthouse_merged[lighthouse_merged["Cluster Labels"]==0]

,Lighthouse Name,Latitude,Longitude,Community,YearBuilt,TowerHeight,FocalHeight,Range,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Estaca de Bares,43.78623,-7.68430,Galicia,1850.0,10.0,101.0,25.0,0,Seafood Restaurant,Spanish Restaurant,None,None,None,None,None,None,None,None
18,Punta Insua,42.77249,-9.12503,Galicia,1921.0,14.0,27.0,15.0,0,Seafood Restaurant,None,None,None,None,None,None,None,None,None


The members of Cluster 1 are characterized by the total absence of venues in its vicinity. They are our candidates to install the new catering business and we will come back to analyze this cluster later.

In [33]:
lighthouse_merged[lighthouse_merged["Cluster Labels"]==1]

,Lighthouse Name,Latitude,Longitude,Community,YearBuilt,TowerHeight,FocalHeight,Range,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Cabo Prior,43.56761,-8.31453,Galicia,1853.0,7.0,107.0,22.0,1,None,None,None,None,None,None,None,None,None,None
6,Cabo Villano,43.16041,-9.21093,Galicia,1896.0,25.0,104.0,28.0,1,None,None,None,None,None,None,None,None,None,None
8,Isla de Sálvora,42.46586,-9.01308,Galicia,1852.0,16.0,40.0,21.0,1,None,None,None,None,None,None,None,None,None,None
11,Islas Sisargas,43.35991,-8.84457,Galicia,1853.0,11.0,110.0,23.0,1,None,None,None,None,None,None,None,None,None,None
16,Punta Candelaria,43.71083,-8.04709,Galicia,1954.0,9.0,89.0,21.0,1,None,None,None,None,None,None,None,None,None,None
20,Punta Nariga,43.32068,-8.91010,Galicia,1998.0,39.0,55.0,22.0,1,None,None,None,None,None,None,None,None,None,None
21,Punta Roncadoira,43.73576,-7.52530,Galicia,1984.0,14.0,94.0,21.0,1,None,None,None,None,None,None,None,None,None,None
22,Touriñán (Toriñana),43.05336,-9.29814,Galicia,1898.0,14.0,65.0,24.0,1,None,None,None,None,None,None,None,None,None,None


Cluster 2 members have at least 3 kinds of venues, mainly Spanish and Tapas Restaurants.

In [34]:
lighthouse_merged[lighthouse_merged["Cluster Labels"]==2]

,Lighthouse Name,Latitude,Longitude,Community,YearBuilt,TowerHeight,FocalHeight,Range,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cabo Corrubedo,42.57627,-9.09009,Galicia,1854.0,14.0,32.0,15.0,2,Tapas Restaurant,Spanish Restaurant,Restaurant,None,None,None,None,None,None,None
10,Isla Pancha,43.55654,-7.04204,Galicia,1859.0,13.0,28.0,21.0,2,Spanish Restaurant,Seafood Restaurant,Restaurant,Pizza Place,Italian Restaurant,Food,Diner,None,None,None
12,La Guía,42.25953,-8.70213,Galicia,1844.0,21.0,37.0,15.0,2,Spanish Restaurant,Seafood Restaurant,Restaurant,Mexican Restaurant,Café,Burger Joint,Breakfast Spot,Brazilian Restaurant,None,None
13,Monte del Faro,42.21412,-8.91571,Galicia,1853.0,10.0,187.0,22.0,2,Spanish Restaurant,Snack Place,Seafood Restaurant,Restaurant,None,None,None,None,None,None
15,Punta Cabalo,42.57242,-8.88399,Galicia,1853.0,5.0,13.0,10.0,2,Spanish Restaurant,Seafood Restaurant,Gastropub,None,None,None,None,None,None,None
17,Punta Frouxeira,43.61806,-8.18836,Galicia,1992.0,30.0,75.0,20.0,2,Spanish Restaurant,Café,Breakfast Spot,None,None,None,None,None,None,None
19,Punta Laxe,43.23214,-9.01121,Galicia,1995.0,11.0,66.0,20.0,2,Tapas Restaurant,Spanish Restaurant,Snack Place,Seafood Restaurant,Restaurant,Food Court,Burger Joint,Bakery,None,None
23,Torre de Hércules,43.38594,-8.40648,Galicia,150.0,49.0,106.0,23.0,2,Spanish Restaurant,Tapas Restaurant,Restaurant,Seafood Restaurant,Gastropub,Food,None,None,None,None


Cluster 3 has only one member. Its most common venue is seafood restaurant and the second is diner.

In [35]:
lighthouse_merged[lighthouse_merged["Cluster Labels"]==3]

,Lighthouse Name,Latitude,Longitude,Community,YearBuilt,TowerHeight,FocalHeight,Range,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Cabo Ortegal,43.77107,-7.87018,Galicia,1954.0,10.0,124.0,18.0,3,Seafood Restaurant,Diner,None,None,None,None,None,None,None,None


Cluster 4 has only one member due to its singular most common venue. It is not usual to find a turkish restaurant nearby a lighthouse in Galicia.

In [36]:
lighthouse_merged[lighthouse_merged["Cluster Labels"]==4]

,Lighthouse Name,Latitude,Longitude,Community,YearBuilt,TowerHeight,FocalHeight,Range,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Cabo Prioriño,43.45879,-8.34033,Galicia,1854.0,5.0,36.0,23.0,4,Turkish Restaurant,None,None,None,None,None,None,None,None,None


Cluster 5 is the most heterogeneous one. All the lighthouses have several venues and they are diverse.

In [37]:
lighthouse_merged[lighthouse_merged["Cluster Labels"]==5]

,Lighthouse Name,Latitude,Longitude,Community,YearBuilt,TowerHeight,FocalHeight,Range,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Cabo Finisterre,42.88236,-9.27196,Galicia,1853.0,17.0,143.0,23.0,5,Seafood Restaurant,Spanish Restaurant,Sandwich Place,Restaurant,Pizza Place,Food,Diner,None,None,None
5,Cabo Silleiro,42.10436,-8.89655,Galicia,1862.0,30.0,85.0,24.0,5,Restaurant,Seafood Restaurant,Cafeteria,None,None,None,None,None,None,None
9,Isla de Ons,42.38244,-8.93639,Galicia,1865.0,12.0,127.0,25.0,5,Food,Tapas Restaurant,Seafood Restaurant,Burger Joint,None,None,None,None,None,None
14,Punta Atalaya (o San Ciprián),43.70050,-7.43680,Galicia,1860.0,14.0,41.0,15.0,5,Seafood Restaurant,Restaurant,Pizza Place,Food,Breakfast Spot,None,None,None,None,None


## Conclusion: the most appropriate lighthouses to open a new food business

As we explained before, lighthouses in Cluster 1 are our best candidates because of its lack of restaurantes and bars nearby. At this point, we remember that the government grant will be proportional to the age of the lighthouse. So, the best places for our business will be, in descending order:

In [38]:
lighthouse_merged[lighthouse_merged["Cluster Labels"]==1].sort_values(by = "YearBuilt")[["Lighthouse Name", "YearBuilt"]]

,Lighthouse Name,YearBuilt
8,Isla de Sálvora,1852.0
3,Cabo Prior,1853.0
11,Islas Sisargas,1853.0
6,Cabo Villano,1896.0
22,Touriñán (Toriñana),1898.0
16,Punta Candelaria,1954.0
21,Punta Roncadoira,1984.0
20,Punta Nariga,1998.0


There are 8 candidate lighthouses to place our business, sorted by its year of construction. The first lighthouse was built more than a century and a half before the last one, so the amount of money received from the Galicia government could be very different in each case.

## Discussion

Despite we had reasonable results for our purpose, it is important to keep in mind that there are many other limitations to build up a new restaurant, bar or any other food business.

For instance, according to the previous table, the preferred lighthouse is \emph{Isla de Sálvora}. But this lighthouse is located on an inhabited island, whose name is Sálvora! It may not be a good a idea to place your food business where there are no people.

## Conclusions and Future Work

In order to avoid this issues, in the future it would be necessary to input more information to the clustering algorithm or even before this stage. 

Other important question is the chose of the number of clusters, which was quite successful this time, according to the results, but could be improved.